# Functional Connectivity Analysis
***

Now we have an idea of three important components to analyzing neuroimaging data:

1. Data manipulation
2. Cleaning and confound regression
3. Parcellation and signal extraction

In this notebook the goal is to integrate these 3 basic components and perform a full analysis of group data using **Intranetwork Functional Connectivity (FC)**. 

Intranetwork functional connectivity is essentially a result of performing correlational analysis on mean signals extracted from two ROIs. Using this method we can examine how well certain resting state networks, such as the **Default Mode Network (DMN)**, are synchronized across spatially distinct regions. 

ROI-based correlational analysis forms the basis of many more sophisticated kinds of functional imaging analysis.

## Notebook Outline
***

The outline of the notebook is divided into two parts. The first part directly uses what you've learned and builds upon it to perform the final functional connectivity analysis on group data. 

The second part shows how we can use Nilearn's convenient wrapper functionality to perform the same task with *significantly less effort*. 

#### Part A: Manual computation 
1. Functional data cleaning and confound regression
2. Applying a parcellation onto the data
3. Computing the correlation between two ROI time-series


#### Part B: Using Nilearn's high-level features
1. Using NiftiLabelsMasker to extract cleaned time-series
2. Computing the correlation between two ROI time-series
3. Performing analysis on all subjects
4. Visualization of final results

In [ ]:
import os
from nilearn import signal as sgl
from nilearn import image as img
from nilearn import plotting as plot
from nilearn import datasets
import nibabel as nib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
sub = '10788'

In [ ]:
#Setting up directories
func_dir = '../data/ds000030/derivatives/fmriprep/sub-{}/func/'.format(sub)

#Pre-processed functional data in MNI space
func_file = 'sub-{}_task-rest_bold_space-MNI152NLin2009cAsym_preproc.nii.gz'.format(sub)
func_path = os.path.join(func_dir,func_file) 

#Confounds.tsv extracted using FMRIPREP 
confound_file = 'sub-{}_task-rest_bold_confounds.tsv'.format(sub)
confound_path = os.path.join(func_dir,confound_file) 

#MNI Brainmask outputted from FMRIPREP
mask_file = 'sub-{}_task-rest_bold_space-MNI152NLin2009cAsym_brainmask.nii.gz'.format(sub)
mask_path = os.path.join(func_dir,mask_file) 

### Part A: Manual Computation of Functional Connectivity
***

#### 1. Cleaning your functional data using filtering, dummy TR removal and confound regression
The first step to any functional analysis is to clean the data

We'll define a function to help extract our confound regressors from the .tsv file for us

In [ ]:
#Refer to part_06 for code + explanation
def extract_confounds(confound_tsv,confounds,dt=True):
    '''
    Arguments:
        confound_tsv                    Full path to confounds.tsv
        confounds                       A list of confounder variables to extract
        dt                              Compute temporal derivatives [default = True]
        
    Outputs:
        confound_mat                    
    '''
    
    #Load in data using Pandas then extract relevant columns
    confound_df = pd.read_csv(confound_tsv,delimiter='\t') 
    confound_df = confound_df[confounds]
    
    #If using temporal derivatives 
    if dt:
        #For each column create a new column '<colname>_dt' containing the step-wise differences
        for col in confound_df.columns:
            confound_df['{}_dt'.format(col)] = confound_df[col].diff() 
    
    #Convert into a matrix of values (timepoints)x(variable)
    confound_mat = confound_df.values 
    
    #Return confound matrix
    return confound_mat
    

Let's clean our data!

In [ ]:
#Load functional image

#Extract confounds + TR drop
 
#Clean functional image


#### 2. Setting up the parcellation scheme

To apply a parcellation we'll have to specify a parcellation to use. 
For this analysis we'll be using a spatially separated version [Yeo 2011 - 7 Networks](https://www.ncbi.nlm.nih.gov/pubmed/21653723).

We chose this parcellation since it nicely characterizes the **DMN**, our network of interest.  

In [ ]:
yeo_atlas = img.load_img('../resources/rois/yeo_2011/Yeo_JNeurophysiol11_MNI152/relabeled_yeo_atlas.nii.gz') 

#Let's visualize it
plot.plot_roi(yeo_atlas,cmap='Paired')

Remember that in order to use the parcellation with our functional data it must have the same dimensions. It turns out that the parcellation schema has slightly different dimensions, so we need to resample: 

In [ ]:
#We use "nearest" to preserve the label values (a label of 2.42 for example, doesn't and shouldn't exist) 


#### 3. Intra-network functional connectivity analysis
Intra-network functional connectivity analysis is determined by computing the correlation between the mean time-series of two spatially distinct regions within the same network. 

To perform this analysis requires a few simple steps:
1. Select 2 ROIs from the same network (DMN) 
2. Extract the mean time-series from both regions 
3. Compute the correlation between the two mean ROI time-series

The first step is to select two ROIs from the DMN. We've already gone through the hassle of selecting these two regions but many possible combinations exist. 

In [ ]:
#Select two ROIs and visualize
source_ROI = 44
target_ROI = 46

In [ ]:
#Visualize the ROI, we can do this by masking our all values not matching our ROI
 

#Apply the mask to the data


#Visualize


In [ ]:
#Do the same for the target ROI

With our two regions selected, we'll now extract the mean time-series for each of our two ROIs

In [ ]:
#Recall that in the parcellation, each voxel is labelled with a number corresponding to a distinct parcel
#We want to extract parcels belonging to our ROI.

#Pull the voxels indices belonging to the ROI 


#Get voxel coordinates (x,y,z) list of source and target ROIs


#Load up functional data to extract ROI voxels from


#Extract the list of voxel time-series belonging to each ROI
#This is now a (roi voxel)x(timepoints) array


#We want to compute the mean timeseries of each list of voxels (source and target) 
#This will be a (1) x (timepoints) vector


Now we've extracted two average time-series, one from the source, and one from the target.
The last step is to compute the correlation between them. This will tell us how well the average time-series from the two DMN regions synchronize with each other.

To do this we use <code>np.corrcoef(x,y)</code> which returns a matrix of form:

$$ \rho=
\left( \begin{matrix}
\rho_{1,1} & \rho_{1,2} \\
\rho_{2,1} & \rho_{2,2}
\end{matrix} \right)
$$

The diagonals represent the correlation of signals with themselves. These are always $1$. The off-diagonal represents the correlation of one signal with another is exactly what we want. In addition the matrix is symmetric so: $\rho_{1,2} = \rho_{2,1}$. 

In [ ]:
#Compute correlation and pull the value in the first row, second column


### Part B: Using nilearn's label masker to extract the timeseries
***
nilearn has a built in function for extracting timeseries from functional files and doing all the extra signal processing at the same time!

In [ ]:
from nilearn import input_data

masker = input_data.NiftiLabelsMasker(labels_img=yeo_atlas,
                                      standardize=True,
                                      memory='nilearn_cache',
                                      verbose=1)

### Using nilearn's ConnectivityMeasure to calculate our correlation matrix

The avalable options are “correlation”, “partial correlation”, “tangent”, “covariance”, “precision” or other utilites in sci-py could be plugged in ([see here for an example](http://nilearn.github.io/auto_examples/03_connectivity/plot_multi_subject_connectome.html#sphx-glr-auto-examples-03-connectivity-plot-multi-subject-connectome-py)).

Let's do correlation this time.

In [ ]:
ctrl_correlation_matrices.shape

Now that we have the full set of connectivity values across multiple ROIs, we'd like to pull matrices specific to the ROI's of interest (44,46). 
We'll first pull the set of intra-network connectivities for our two ROIs of interest. 

In [ ]:
from nilearn.connectome import ConnectivityMeasure

connectivity_metric = 'correlation'
connectivity_measure = ConnectivityMeasure(kind=connectivity_metric)

In [ ]:
import bids

In [ ]:
layout = bids.layout.BIDSLayout('../data/ds000030')

In [ ]:
subjects = layout.get_subjects()

In [ ]:
pooled_subjects = []
ctrl_subjects = []
schz_subjects = []

for sub in subjects:
    func_file = layout.get(subject='10171', modality='func', type='preproc', extensions='nii.gz', return_type='file')[0]
    confound_file = layout.get(subject='10171', modality='func', type='confounds', return_type='file')[0]
    func_img = img.load_img(func_file)
    func_img = func_img.slicer[:,:,:,5:]
    confounds = extract_confounds(confound_file,['X','Y','Z','RotX','RotY','RotZ','GlobalSignal'])
    confounds = confounds[5:,:] 
    time_series = masker.fit_transform(func_img, confounds)
    pooled_subjects.append(time_series)
    if sub.startswith('1'):
        ctrl_subjects.append(time_series)
    if sub.startswith('5'):
        schz_subjects.append(time_series)

In [ ]:
import seaborn as sns

Seaborn visualization is best achieved using a Pandas dataframe (much like R's dataframes). A dataframe is essentially a spreadsheet-like table and is easily created using numpy arrays.

In [ ]:
correlation_measure = ConnectivityMeasure(kind='correlation')

In [ ]:
ctrl_correlation_matrices = correlation_measure.fit_transform(ctrl_subjects)
schz_correlation_matrices = correlation_measure.fit_transform(schz_subjects)

In [ ]:
mean_correlation_matrix = correlation_measure.mean_

In [ ]:
import numpy as np
import matplotlib.pylab as plt
from nilearn import plotting

def plot_matrices(matrices, matrix_kind):
    n_matrices = len(matrices)
    fig = plt.figure(figsize=(n_matrices * 4, 4))
    for n_subject, matrix in enumerate(matrices):
        plt.subplot(1, n_matrices, n_subject + 1)
        matrix = matrix.copy()  # avoid side effects
        # Set diagonal to zero, for better visualization
        np.fill_diagonal(matrix, 0)
        vmax = np.max(np.abs(matrix))
        title = '{0}, subject {1}'.format(matrix_kind, n_subject)
        plotting.plot_matrix(matrix, vmin=-vmax, vmax=vmax, cmap='RdBu_r',
                             title=title, figure=fig, colorbar=False)


In [ ]:
plot_matrices(ctrl_correlation_matrices, 'correlation')

In [ ]:
plot_matrices(schz_correlation_matrices, 'correlation')

In [ ]:
ctrl_mean_correlation_matrix = ctrl_correlation_matrices.mean(axis=0)
schz_mean_correlation_matrix = schz_correlation_matrices.mean(axis=0)

In [ ]:
ctrl_mean_correlation_matrix[43, 45]

In [ ]:
schz_mean_correlation_matrix[43,45]